In [124]:
from models import models
from datetime import datetime, timedelta
import time
from tortoise import Tortoise
from data import config
from tortoise.queryset import Q
import requests
from pprint import PrettyPrinter
import aiohttp
import asyncio
from models import models

pp = PrettyPrinter(indent=4)

In [147]:

async def check_transactions():
    await Tortoise.init(config=config.TORTOISE_ORM)
    params = {"address": "EQDcYqGh0d6i_kogT4QXvTuEq_iu61qPrZDVhbGnDvdrk-f3",
            "limit": 20}
    response = None
    for i in range(5):
        async with aiohttp.ClientSession() as session:
            async with session.get(url="https://toncenter.com/api/v2/getTransactions", params=params) as resp:
                if resp.status != 200:
                    await asyncio.sleep(1)
                    continue
                else:
                    response = await resp.json()
                    break
    if response is None:
        return
    if response['ok'] is True:
        transactions = response['result']
        pp.pprint(transactions[0])
        for transaction in transactions:
            if transaction['transaction_id']['hash'] == "KfOUWLY6Hj80BjW533q8WOeABF1JumhlRU1Ua32ABJo=":
                return
            if transaction['in_msg']['value'] == '0':
                continue
            
            amount = int(transaction['in_msg']['value']) / 1000000000
            hash = transaction['transaction_id']['hash']
            wallet = transaction['in_msg']['source']
            code = transaction['in_msg']['message']

            existing_transaction = await models.UserBalanceChange.get_or_none(hash=hash)
            if existing_transaction:
                continue

            user_balance_change = None
            if code != '':
                user_balance_change = await models.UserBalanceChange.get_or_none(code=code, state="created")
            
            if user_balance_change is not None:
                user = await user_balance_change.user
                user_balance_change.amount = amount
                user_balance_change.state = "done"
                user_balance_change.hash = hash
                user_balance_change.wallet = wallet
                user.balance += amount
                await user_balance_change.save()
                await user.save()
            else:
                await models.UserBalanceChange.create(user=None,
                                                     type="topup",
                                                     amount=amount,
                                                     hash=hash,
                                                     wallet=wallet,
                                                     code=code,
                                                     state="done")

await check_transactions()

{   '@type': 'raw.transaction',
    'data': 'te6cckECCgEAAkoAA7V9xioaHR3qL+SiBPhBe9O4Sr+K7rWo+tkNWFsacO92uTAAAa/IHQaQEX4PpxGfroxra2emyL5ar6mYYX7te3aIXqD8FUXkXhKQAAGpPq7ksDYtWt0wADRqijZoAQIDAgHgBAUAgnJ9yO4R1EThEoDVf+sk/cudO7iI38j6LNubNBFvs/DYtif3Gu3Sx1X1zgXO2W1u9uBWwu4DeQR+Z4JO99XdylJKAhMMwWf4xhmTzwRACAkB4YgBuMVDQ6O9RfyUQJ8IL3p3CVfxXda1H1shqwtjTh3u1yYEGpS5m3qgvH++qMxFUxQpPnyrR2HEk21MNa4yIZg8w5AfUhqOIKGcziTFZdXiDPwVeFf9EKIiEeCWiWHvyAdoYU1NGLsWrXBwAAAAMAAcBgEB3wcAaEIAc/t65WWeBvqaiKSagnsZIvoElpmSaCeJ2hatLe5JYmsiy0F4AAAAAAAAAAAAAAAAAAAAsUgBuMVDQ6O9RfyUQJ8IL3p3CVfxXda1H1shqwtjTh3u1ycAOf29crLPA31NRFJNQT2MkX0CS0zJNBPE7QtWlvcksTWRZaC8AAYUWGAAADX5A6DSBMWrW6ZAAJ1BnYMTiAAAAAAAAAAAEQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAG/Jh6EgTBRYQAAAAAAAAgAAAAAAA0R/fECkSVgVmiUuk5myW/WtCKiUoKPKf3iefufHmQT+QFAWDCvvv38=',
    'fee': '6192611',
    'in_msg': {   '@type': 'raw.message',
                  'body_hash': 'iv0xbeSxwRSL/tqT3UiyhUMPGLPQVncH1iqCNml34y8=',
     

In [34]:
1000000 / 1000000000

0.001